In [1]:

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

pd.set_option(
    "display.max_columns", None
)  # to display the total number columns present in the dataset

/Users/mg_0328/anaconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
import plotly.express as px

df = px.data.gapminder().query("country == 'Canada'")
df

country continent  year  lifeExp       pop    gdpPercap iso_alpha  iso_num
240  Canada  Americas  1952   68.750  14785584  11367.16112       CAN      124
241  Canada  Americas  1957   69.960  17010154  12489.95006       CAN      124
242  Canada  Americas  1962   71.300  18985849  13462.48555       CAN      124
243  Canada  Americas  1967   72.130  20819767  16076.58803       CAN      124
244  Canada  Americas  1972   72.880  22284500  18970.57086       CAN      124
245  Canada  Americas  1977   74.210  23796400  22090.88306       CAN      124
246  Canada  Americas  1982   75.760  25201900  22898.79214       CAN      124
247  Canada  Americas  1987   76.860  26549700  26626.51503       CAN      124
248  Canada  Americas  1992   77.950  28523502  26342.88426       CAN      124
249  Canada  Americas  1997   78.610  30305843  28954.92589       CAN      124
250  Canada  Americas  2002   79.770  31902268  33328.96507       CAN      124
251  Canada  Americas  2007   80.653  33390141  36319.23501       CAN      124

## Stacked 100 percent _chart working code

In [ ]:
def stacked_100_percent_chart(dataframe, par, var):
    import plotly.graph_objects as go
    import pandas as pd
    import warnings

    warnings.filterwarnings('ignore')
    import plotly.io as pio
    import os
    import time
    start_time = time.time()
    # Collecting the output summary for flyout
    flyout_result = {'Execution Time': None}

    '''
        About: 
          For the same data, we can create stacked 100 percent chart in the same figure to get more information.       
        Points to Note:
          1) Stacked 100 percent is not suitable for continuous variables. 
          2) If X is categorical, then Y should be numeric and vice versa. 

        Documentation - 
        X = Enter the column name to position marks along X-axis. Column Name. 
        Y = Enter the column name to position marks along Y-axis. Column Name.   
        Title = Enter a title for the stacked 100 percent chart. If not entered, a default value will be added to the figure. Text. 
        X_label = Enter a suitable name for labeling X-axis. If not entered, a default value will be added to the figure. Text.
        Y_label = Enter a suitable name for labeling Y-axis. If not entered, a default value will be added to the figure. Text. 
        '''


    # Extract parameters
    x_axis = par.get("X-Axis")
    y_axis = par.get("Y-Axis")

    # Title / O
    title = par.get("Title", "")
    if title == "":
        title = "Stacked 100 Percent Chart"

    # X Label / O
    x_label = par.get("X-label", "")
    if x_label == "":
        x_label = str(x_axis)
    # Y Label / O
    y_label = par.get("Y-label", "")
    if y_label == "":
        y_label = str(', '.join(y_axis))

    file_name = 'image_stacked_100_percent_chart_plotly_APRIL_3_testing.json'

    ## Exceptions

    if not x_axis or not y_axis or not isinstance(x_axis, str) or not isinstance(y_axis, list) or not y_axis:
        raise Exception("X-Axis and Y-Axis should be specified with at least one column each.")

    if "Title" in par and not isinstance(par["Title"], str):
        raise ValueError("Plot title must be provided as a string.")

    if dataframe[x_axis].isnull().any():
        raise ValueError("Null values present in y-axis column")

    for column in y_axis:
        if dataframe[column].isnull().any():
            raise ValueError("Null values present in x-axis column")


    # Extract categories and values from the DataFrame
    categories = dataframe[x_axis].tolist()
    values_list = [dataframe[column].tolist() for column in y_axis]
    legend_labels = y_axis

    # Normalize the data to create percentages
    total_values = [sum(values) for values in zip(*values_list)]
    percentages = [[round(value / total, 2) for value, total in zip(values, total_values)] for values in values_list]

    # Create stacked 100 percent chart
    fig = go.Figure()

    for i, values in enumerate(percentages):
        fig.add_trace(go.Bar(
            text = values,
            x=categories,
            y=values,
            name=legend_labels[i]
        ))
    # Removing textfont data label color as plotly assign it automatically
    # fig.update_traces(textfont={"color": "black"})
    fig.update_xaxes(showgrid=False)
    fig.update_yaxes(showgrid=True, tickformat="0.0%")

    # Update layout to have 100% y-axis range
    fig.update_layout(
        barmode='stack',
        yaxis=dict(
            title=y_label
        ),
        xaxis=dict(title=x_label),
        title=title,
        title_x=0.5,
        legend=dict(x=0, y=-0.1, orientation='h')
    )

    fig.update_layout(paper_bgcolor="black", font_color="white")

    pio.write_json(fig, file=os.path.join(var["outputFileDirectory"], file_name), pretty=True)
    # Computing the time for execution as a table
    ex_time = round((time.time() - start_time), 4)
    if ex_time > 60:
        ex_minutes = int(ex_time / 60)
        ex_sec = round(ex_time % 60, 4)
        ex_time_refined = str(ex_minutes) + ' minutes ' + str(ex_sec) + ' sec'
    else:
        ex_time_refined = str(ex_time) + ' sec'

    execution_time = {}
    execution_time['Time'] = ex_time_refined
    execution_time_df = pd.DataFrame.from_dict(execution_time, orient='index', columns=['Value'])

    flyout_result['X-axis'] = x_axis
    flyout_result['Y-axis'] = ', '.join(y_axis)
    flyout_result['Title'] = title
    flyout_result['X Label'] = x_label if x_label is not None else 'None'
    flyout_result['Y Label'] = y_label if y_label is not None else 'None'
    function_summary = pd.DataFrame.from_dict(flyout_result, orient='index', columns=['Value'])

    output_flyout = {"Input Parameter Selection": function_summary[1:],
                     "Function Execution Time": execution_time_df}

    return {'TEXT': output_flyout}

import plotly.express as px

df = px.data.gapminder().query("country == 'Canada'")
print(df.head())

par={"X-Axis":"lifeExp","Y-Axis":["pop","iso_num"]}
var={"outputFileDirectory":"/Users/mg_0328/Downloads/PLOTS/plotly-poc/src/json_plotly/json_plotly_testing"}

print(stacked_100_percent_chart(df,par,var))

In [4]:
from gekko import GEKKO    
m = GEKKO(remote=False)

# create variables
x1,x2,x3,x4 = m.Array(m.Var,4,lb=1,ub=5)

# initial values
x1.value = 1; x2.value = 5; x3.value = 5; x4.value = 1

m.Equation(x1*x2*x3*x4>=25)
m.Equation(x1**2+x2**2+x3**2+x4**2==40)
m.Minimize(x1*x4*(x1+x2+x3)+x3)
m.solve(disp=False)

print('Results')
print('x1: ' + str(x1.value[0])); print('x2: ' + str(x2.value[0]))
print('x3: ' + str(x3.value[0])); print('x4: ' + str(x4.value[0]))

Results
x1: 1.0
x2: 4.742999637
x3: 3.8211499845
x4: 1.3794082931


In [33]:
import pandas as pd
from sklearn.model_selection import train_test_split

# for one hot encoding with feature-engine
from sklearn.preprocessing import OneHotEncoder

In [34]:
d={"A":[1,6,7,2,5,7,3,8,3,2,4,6,4,6,8,9,1,0,3,6],"B":["A","F","B","B","V",
                                                      "B","D","R","R","F",
                                                      "G","DD","D","Z","V",
                                                      "D","T","R","W","P"]}

In [35]:
d=pd.DataFrame(d)
d

A   B
0   1   A
1   6   F
2   7   B
3   2   B
4   5   V
5   7   B
6   3   D
7   8   R
8   3   R
9   2   F
10  4   G
11  6  DD
12  4   D
13  6   Z
14  8   V
15  9   D
16  1   T
17  0   R
18  3   W
19  6   P

In [36]:
d["B"].value_counts()

B
B     3
D     3
R     3
F     2
V     2
A     1
G     1
DD    1
Z     1
T     1
W     1
P     1
Name: count, dtype: int64

In [37]:
ohe=OneHotEncoder(handle_unknown="infrequent_if_exist",max_categories=5,sparse_output=False)

In [38]:
ohe.fit(d[["B"]])

OneHotEncoder(handle_unknown='infrequent_if_exist', max_categories=5,
              sparse_output=False)

In [40]:
df=ohe.transform(d[["B"]])
df

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]])

In [41]:
df=pd.DataFrame(df,columns=ohe.get_feature_names_out())
df

B_B  B_D  B_R  B_V  B_infrequent_sklearn
0   0.0  0.0  0.0  0.0                   1.0
1   0.0  0.0  0.0  0.0                   1.0
2   1.0  0.0  0.0  0.0                   0.0
3   1.0  0.0  0.0  0.0                   0.0
4   0.0  0.0  0.0  1.0                   0.0
5   1.0  0.0  0.0  0.0                   0.0
6   0.0  1.0  0.0  0.0                   0.0
7   0.0  0.0  1.0  0.0                   0.0
8   0.0  0.0  1.0  0.0                   0.0
9   0.0  0.0  0.0  0.0                   1.0
10  0.0  0.0  0.0  0.0                   1.0
11  0.0  0.0  0.0  0.0                   1.0
12  0.0  1.0  0.0  0.0                   0.0
13  0.0  0.0  0.0  0.0                   1.0
14  0.0  0.0  0.0  1.0                   0.0
15  0.0  1.0  0.0  0.0                   0.0
16  0.0  0.0  0.0  0.0                   1.0
17  0.0  0.0  1.0  0.0                   0.0
18  0.0  0.0  0.0  0.0                   1.0
19  0.0  0.0  0.0  0.0                   1.0

In [42]:
d

A   B
0   1   A
1   6   F
2   7   B
3   2   B
4   5   V
5   7   B
6   3   D
7   8   R
8   3   R
9   2   F
10  4   G
11  6  DD
12  4   D
13  6   Z
14  8   V
15  9   D
16  1   T
17  0   R
18  3   W
19  6   P

In [46]:
top_categories=5
cols=["B"]

In [47]:
ohe=OneHotEncoder(handle_unknown="infrequent_if_exist",max_categories=top_categories,sparse_output=False)
ohe.fit(d[cols])
df=ohe.transform(d[cols])
df=pd.DataFrame(df,columns=ohe.get_feature_names_out())
display(df)

B_B  B_D  B_R  B_V  B_infrequent_sklearn
0   0.0  0.0  0.0  0.0                   1.0
1   0.0  0.0  0.0  0.0                   1.0
2   1.0  0.0  0.0  0.0                   0.0
3   1.0  0.0  0.0  0.0                   0.0
4   0.0  0.0  0.0  1.0                   0.0
5   1.0  0.0  0.0  0.0                   0.0
6   0.0  1.0  0.0  0.0                   0.0
7   0.0  0.0  1.0  0.0                   0.0
8   0.0  0.0  1.0  0.0                   0.0
9   0.0  0.0  0.0  0.0                   1.0
10  0.0  0.0  0.0  0.0                   1.0
11  0.0  0.0  0.0  0.0                   1.0
12  0.0  1.0  0.0  0.0                   0.0
13  0.0  0.0  0.0  0.0                   1.0
14  0.0  0.0  0.0  1.0                   0.0
15  0.0  1.0  0.0  0.0                   0.0
16  0.0  0.0  0.0  0.0                   1.0
17  0.0  0.0  1.0  0.0                   0.0
18  0.0  0.0  0.0  0.0                   1.0
19  0.0  0.0  0.0  0.0                   1.0